# PRTECAN Module Deep Dive Tutorial

This tutorial explores the `prtecan` module for processing Tecan plate reader data with a focus on:
- File parsing and data structures
- Titration curve analysis
- Advanced fitting capabilities
- Visualization techniques

In [ ]:
# Magic commands for development
%load_ext autoreload
%autoreload 2

# Setup
from pathlib import Path

import matplotlib.pyplot as plt

from clophfit import prtecan

# Configure notebook
%matplotlib inline
plt.style.use("seaborn-v0_8")
data_dir = Path("../../tests/Tecan/140220")

## 4. Fitting Framework

test:

- E10
- F10
- G09

TODO:

- Remove datapoint ini fin outlier

In [ ]:
data_dir

In [ ]:
data_dir = Path("../../tests/Tecan/140220")
data_dir = Path("/home/dati/arslanbaeva/data/raw/L2/")

tit = prtecan.Titration.fromlistfile(data_dir / "list.pH.csv", is_ph=1)
tit.load_additions(data_dir / "additions.pH")
tit.load_scheme(data_dir / "scheme.txt")
tit.params.bg_mth = "meansd"
tit.bg_err

In [ ]:
tit.result_global.compute_all()

In [ ]:
tit.result_global.plot_k()

In [ ]:
def output_fr(fr):
    print(fr.result.redchi)
    print(fr.dataset)
    return fr.figure

In [ ]:
import arviz as az

from clophfit.binding.fitting import (
    fit_binding_glob,
    fit_binding_glob_reweighted,
    fit_binding_pymc,
    fit_binding_pymc_compare,
    outlier2,
)

In [ ]:
k = "A01"

ds = tit._create_global_ds(k)
ds["y1"].y_err.mean(), ds["y2"].y_err.mean()
ds

In [ ]:
fit_binding_glob(ds).figure

In [ ]:
ds["y1"].y_errc

In [ ]:
fr = outlier2(ds, k, plot_z_scores=1, threshold=3.0)
fr.figure

In [ ]:
fr.result.logger = 1

In [ ]:
print(fr.result.redchi)
print(fr.result.chisqr)
fr.dataset

In [ ]:
ds2 = tit._create_ds(k, 2)
ds2

In [ ]:
fit_binding_glob(ds2).figure

In [ ]:
k = "A01"
fr = tit.result_global[k]
fr.figure

In [ ]:
{"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()}

In [ ]:
n_sd = 0.15 / fr.result.params["K"].stderr
print(n_sd)

# Run the model with a single noise scaling factor
trace_single = fit_binding_pymc_compare(
    fr, n_sd=max(n_sd, 1), n_xerr=0.682, learn_separate_y_mag=False
)

# Run the model with separate noise scaling factors for each label
trace_separate = fit_binding_pymc_compare(
    fr, n_sd=max(n_sd, 1), n_xerr=0.682, learn_separate_y_mag=True
)
trace_separate_shot = fit_binding_pymc_compare(
    fr,
    {"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()},
    n_sd=max(n_sd, 1),
    n_xerr=0.682,
    learn_separate_y_mag=True,
)

In [ ]:
az.summary(trace_separate_shot)

In [ ]:
az.summary(trace_single)

In [ ]:
az.summary(trace_separate)

In [ ]:
# You can pass the traces directly to az.compare
comparison_results = az.compare(
    {"single_y_mag": trace_separate_shot, "separate_y_mag": trace_separate}
)

# The result is a pandas DataFrame.
# The best model has the lowest 'loo' or 'waic' value.
# The 'd_loo' column shows the difference from the best model.
comparison_results

In [ ]:
n_sd = 0.15 / fr.result.params["K"].stderr
print(n_sd)
fr_mcmc2 = fit_binding_pymc(fr, n_sd=max(n_sd, 1), n_xerr=0.682)

In [ ]:
import arviz as az

df = az.summary(fr_mcmc2.mini)
fr_mcmc2.figure

In [ ]:
df

In [ ]:
import arviz as az

az.summary(fr_mcmc2.mini)

In [ ]:
# da1.mask = ~outlier_glob(fr.result.residual, plot_z_scores=1, threshold=2)

In [ ]:
output_fr(fit_binding_glob_reweighted(ds, k, threshold=2.25))